In [22]:
import QuantLib as ql
import numpy as np

# Market inputs
spot = 29.855  # USD/TWD
today = ql.Date(10, 8, 2025)
ql.Settings.instance().evaluationDate = today
dc = ql.Actual365Fixed()

expiry = today + ql.Period('3M')  # 3M
T = dc.yearFraction(today, expiry)
    
rd = 0.00826   # domestic (TWD)
disc_rd = np.exp(-rd * T)  # discount factor for domestic rate
rf = 0.0450    # foreign (USD)
disc_rf = np.exp(-rf * T)  # discount factor for foreign rate

# ATM, RR, BF quotes (example)
atm_vol = 0.08
rr_25 = 0.0020
bf_25 = 0.0010

# Build vols for 25Δ call/put
vol_25_call = atm_vol + bf_25 + 0.5 * rr_25
vol_25_put  = atm_vol + bf_25 - 0.5 * rr_25
print("vol_25_call:", vol_25_call)
print("vol_25_put:", vol_25_put)

# Yield term structures
rd_curve = ql.FlatForward(today, rd, dc)
rf_curve = ql.FlatForward(today, rf, dc)

# === Use BlackDeltaCalculator to get strike from delta ===
# Convention: FX Spot Delta, Call = true, premium-adjusted = false
delta_type = ql.DeltaVolQuote.Spot

# 25Δ Call strike
calc_call = ql.BlackDeltaCalculator(ql.Option.Call, delta_type,
                                    spot, disc_rd, disc_rf, vol_25_call)
k_25_call = calc_call.strikeFromDelta(0.25)

# 25Δ Put strike
calc_put = ql.BlackDeltaCalculator(ql.Option.Put, delta_type,
                                   spot, disc_rd, disc_rf, vol_25_put)
k_25_put = calc_put.strikeFromDelta(-0.25)  # put delta is negative

print("25Δ Call Strike:", k_25_call)
print("25Δ Put  Strike:", k_25_put)

vol_25_call: 0.082
vol_25_put: 0.08
25Δ Call Strike: 31.34417136842089
25Δ Put  Strike: 28.135958785205133


In [ ]:
# validate delta-strike relationship
# ===== 1. Market data =====
strike1 = 31.34417136842089 # 25Δ call strike from ql.BlackDeltaCalculator.strikeFromDelta(), this seems incorrect
strike2 = 30.427244289063378 # 25Δ call strike, this is correct

# ===== 2. Evaluation date =====
ql.Settings.instance().evaluationDate = today

# ===== 3. Day count convention =====
day_count = ql.Actual365Fixed()
cal_usd = ql.UnitedStates(ql.UnitedStates.FederalReserve)
cal_twd = ql.Taiwan(ql.Taiwan.TSEC)

# ===== 4. Curves =====
vol_ts = ql.BlackConstantVol(today, cal_usd, vol_25_call, day_count)

# ===== 5. Payoff and exercise =====
payoff1 = ql.PlainVanillaPayoff(ql.Option.Call, strike1)
payoff2 = ql.PlainVanillaPayoff(ql.Option.Call, strike2)
exercise = ql.EuropeanExercise(expiry)

# ===== 6. FX Black-Scholes process (Garman-Kohlhagen) =====
spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot))
domestic_handle = ql.YieldTermStructureHandle(rd_curve)
foreign_handle = ql.YieldTermStructureHandle(rf_curve)
vol_handle = ql.BlackVolTermStructureHandle(vol_ts)

process = ql.BlackScholesMertonProcess(spot_handle,
                                       foreign_handle,
                                       domestic_handle,
                                       vol_handle)

# ===== 7. Pricing engine =====
engine = ql.AnalyticEuropeanEngine(process)

# ===== 8. Option object =====
option = ql.EuropeanOption(payoff1, exercise)
option2 = ql.EuropeanOption(payoff2, exercise)
option.setPricingEngine(engine)
option2.setPricingEngine(engine)

# ===== 9. Result =====
Delta = option.delta()
Delta2 = option2.delta()
print(f"FX Call Option Delta (Strike 1): {Delta}")
print(f"FX Call Option Delta (Strike 2): {Delta2}") 

FX Call Option Delta (Strike 1): 0.08182871631450198
FX Call Option Delta (Strike 2): 0.2500000000000005


In [ ]:
import math
from scipy.stats import norm

# function to do validation
def compute_strike_from_delta(F, sigma, T, delta, r_for=0.0, option_type='Call', delta_type='spot'):
    """
    F: forward price
    sigma: vol (annual)
    T: time in years
    delta: target delta (positive for call e.g. 0.25, absolute)
    r_for: foreign continuous rate (used to convert spot->forward delta if needed)
    option_type: 'Call' or 'Put'
    delta_type: 'forward' or 'spot'
    Returns strike K.
    """
    phi=1
    if option_type == 'Put':
        phi = -1
        if delta > 0:
            raise ValueError("For Put options, delta should be negative (e.g. -0.25)")
    elif option_type == 'Call':
        phi = 1
        if delta < 0:
            raise ValueError("For Call options, delta should be positive (e.g. 0.25)")
    else:
        raise ValueError("option_type must be 'Call' or 'Put'")
    
    # Determine N(d1)
    if delta_type == 'forward':
        Nd1 = phi * delta
    elif delta_type == 'spot':
        # spot delta = e^{-r_f T} * N(d1)  => N(d1) = spot_delta * e^{r_f T}
        Nd1 = phi * delta * math.exp(r_for * T)
    else:
        raise ValueError("delta_type must be 'forward' or 'spot'")

    d1 = norm.ppf(Nd1)  # inverse Gaussian CDF
    # K = F * exp(-d1*sigma*sqrtT + 0.5*sigma^2*T)
    K = F * math.exp(-phi * d1 * sigma * math.sqrt(T)  + 0.5 * sigma * sigma * T)

    return K


F = spot * math.exp((rd - rf) * T)  # forward price
# compute strikes corresponding to 25Δ put and call
K_25C = compute_strike_from_delta(F, vol_25_call, T, 0.25, rf,'Call', 'spot')
K_25P = compute_strike_from_delta(F, vol_25_put, T, -0.25, rf, 'Put', 'spot')

print("25Δ Call Strike:", K_25C)
print("25Δ Put  Strike:", K_25P)

25Δ Call Strike: 30.427244289063378
25Δ Put  Strike: 28.822822711038622
